In [39]:
'065676'

'065676'

In [1]:
# ! pip install jugaad_data, nsetools
from jugaad_data.nse import stock_df
from nsetools import Nse

from datetime import date, datetime, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as fplt
import plotly.graph_objects as go
from plotly import colors as plotly_colors

from multiprocessing import Pool

import requests
from bs4 import BeautifulSoup
import random

from os import listdir
from os.path import isfile, join

import json
import warnings
import math
from random import sample


pd.set_option('display.max_rows', 100)

In [2]:
def get_stocks(name):
    try:
        df = stock_df(symbol=name, from_date = current_date - timedelta(days = 500), to_date = current_date, series="EQ").drop(drop,axis=1)
        df['DATE'] = pd.to_datetime(df['DATE'])
        ID, NAME, _ = all_stocks[name].split('_')
        save = f"./data/{ID}_{NAME}_{str(current_date)}.csv"
        df.to_csv(save,index=None)
    except Exception as e:
        print(e)
        pass


 
path = './data'
AS = AnalyseStocks(check_fresh = False)
all_stocks = AS.all_stocks

drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
current_date = date.today()
stocks = set(all_stocks.keys()) - set([i.split('_')[0] for i in listdir('./data')]) 




pool = Pool(4)
results = pool.map(get_stocks,stocks)
pool.close()
pool.join()

NameError: name 'AnalyseStocks' is not defined

# Code

In [3]:
class DataHandler:
    def __init__(self, data_path = './data', check_fresh = True):
        self.present = date.today()
        self.week_num = self.present.strftime("%W")
        
        self.data_path = data_path
        
        if check_fresh:
            self.__fresh()
        self.data = self.read_data()
        self.all_stocks = self.read_data()['all_stocks']
        
         
    def __fresh(self,):
        files = listdir(self.data_path)
        if not len(files):
            raise Exception(f"No CSV data files present at {self.data_path} Download new data for analysis")

        self.data = self.read_data()
        for file in files:
            key, _ , _ = file.split('_')
            self.data['all_stocks'][key] = file
        self.update_data(self.data)
  
    
    def read_data(self, path = './', file = 'data.json'):
        '''
        Write the data in json file
        args:
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file)) as f:
            return json.load(f)


    def update_data(self, updated_data:dict, path:str = './', file:str = 'data.json'):
        '''
        Update the data in the json file
        args:
            updated_data: Dictonary you want to update
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file), 'w') as f:
            json.dump(updated_data,f)
            return True 
    
    
    def open_live_stock_data(self,name:str):
        '''
        Open the fresh stock from the market
        args:
            name: ID of the stock given
        '''
        drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
        return stock_df(symbol=name, from_date = self.present - timedelta(days = 500), to_date = self.present, series="EQ").drop(drop,axis=1)
    
    
    def open_downloaded_stock(self,name:str):
        '''
        Open the Individual stock based on it's Official Term
        args:
            name: Name / ID given to the stock. Example, Infosys is "INFY"
        returns: DataFrame of that stock
        '''
        return pd.read_csv(join(self.data_path,self.all_stocks[name]))

In [19]:
class AnalyseStocks(DataHandler):
    def __init__(self, check_fresh = True):
        '''
        args:
            path: Path where all the stock files are saved
        '''
        super().__init__(check_fresh = check_fresh)
        self.eligible = {}
        self.registered_stocks = self.read_data()['registered_stocks']
        self.colors = self.read_data()['colors']
        self.rising = {}
    

    def is_ma_eligible(self, df, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH'),limit:float=11):
        '''
        Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            limit: Limit difference between average and low/high threshold for selecting stocks
        '''
        Date, Open, Close, Low, High = names
        Average = f'{str(mv)}-SMA'
        
        stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
        stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()
        
        last_traded = stocks.iloc[-1,:]
        low, high, open_ , close, avg, symbol = last_traded[Low],last_traded[High],last_traded[Open],last_traded[Close], last_traded[Average], last_traded['SYMBOL']
        
        if close < avg or close < open_ : # if red candle or below Average Line, Discard
            return False
        
        diff = min(abs(low - avg), (abs(high - avg)), abs(open_ - avg), abs(close - avg))
        if diff <= limit:
            self.rising[symbol] = stocks.iloc[-mv//2:-1,-1].mean() < avg  # whether the moving average is Upward or Downward according to last 44 days moving averages
            return {symbol : round(diff,2)}

        return False
    
    
    def Ichimoku_Cloud(self,df):
        '''
        Get the values of Lines for Ichimoku Cloud
        args:
            df: Dataframe
        '''
        d = df.sort_index(ascending=False)

        # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
        period9_high = d['HIGH'].rolling(window=9).max()
        period9_low = d['LOW'].rolling(window=9).min()
        tenkan_sen = (period9_high + period9_low) / 2


        # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
        period26_high = d['HIGH'].rolling(window=26).max()
        period26_low = d['LOW'].rolling(window=26).min()
        kijun_sen = (period26_high + period26_low) / 2

        # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
        senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

        # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
        period52_high = d['HIGH'].rolling(window=52).max()
        period52_low = d['LOW'].rolling(window=52).min()
        senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

        # The most current closing price plotted 22 time periods behind (optional)
        chikou_span = d['CLOSE'].shift(-22) # Given at Trading View.

        d['blue_line'] = tenkan_sen
        d['red_line'] = kijun_sen
        d['cloud_green_line_a'] = senkou_span_a
        d['cloud_red_line_b'] = senkou_span_b
        d['lagging_line'] = chikou_span
        return d.sort_index(ascending=True)
    
    
    def update_eligible(self):
        '''
        Save all Eligible stocks for the current week
        '''
        for key in self.registered_stocks:
            result = self.is_ma_eligible(self.open_downloaded_stock(key))
            if result:
                self.eligible.update(result)
        return self.eligible
            
    
    def plot_candlesticks(self,df, names = ('DATE','OPEN','CLOSE','LOW','HIGH'), mv:list = [44,100,200]):
        '''
        Plot a candlestick on a given dataframe
        args:
            df: DataFrame
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            mv: Moving Averages
        '''
        stocks = df.copy()
        Date, Open, Close, Low, High = names
        colors = sample(self.colors,len(mv))
        stocks.sort_index(ascending=False, inplace = True)  # Without reverse, recent rolling mean will be either NaN or equal to the exact value
    

        candle = go.Figure(data = [go.Candlestick(x = stocks[Date], name = 'Trade',
                                                       open = stocks[Open], 
                                                       high = stocks[High], 
                                                       low = stocks[Low], 
                                                       close = stocks[Close]),])
        for i in range(len(mv)):
            stocks[f'{str(mv[i])}-SMA'] = stocks[Close].rolling(mv[i], min_periods = 1).mean()
            candle.add_trace(go.Scatter(name=f'{str(mv[i])} MA',x=stocks[Date], y=stocks[f'{str(mv[i])}-SMA'], 
                                             line=dict(color=colors[i], width=1.1)))

        candle.update_xaxes(
            title_text = 'Date',
            rangeslider_visible = True,
            rangeselector = dict(
                buttons = list([
                    dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                    dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                    dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                    dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                    dict(step = 'all')])))

        candle.update_layout(autosize = False, width = 1400, height = 600,
                             title = {'text': f"{stocks['SYMBOL'][0]} | {self.all_stocks[stocks['SYMBOL'][0]]}",'y':0.97,'x':0.5,
                                      'xanchor': 'center','yanchor': 'top'},
                             margin=dict(l=30,r=30,b=30,t=30,pad=2),
                             paper_bgcolor="lightsteelblue",)

        candle.update_yaxes(title_text = 'Price in Rupees', tickprefix = u"\u20B9" ) # Rupee symbol
        candle.show()

# Investing

In [20]:
class Investing(AnalyseStocks):
    def __init__(self,rolling_mean:int=44):
        '''
        args:
            rolling_mean: Rolling Simple Mean to calulate
        '''
        super().__init__()
        self.rm = rolling_mean
        self._eligible = self.update_eligible()
        self.data = self.read_data()
        self.indices = {'nifty_50': 'Nifty 50','nifty_100':'Nifty 100','nifty_200':'Nifty 200','nifty_500':'Nifty 200'}
      

    def highlight_falling(self, s, column:str):
        '''
        Highlight The rows where average is falling
        args:
            s: Series
            column: Column name(s)
        '''
        is_max = pd.Series(data=False, index=s.index)
        is_max[column] = s.loc[column] == True
        return ['' if is_max.any() else 'background-color: #f7a8a8' for v in is_max]
    
    
    def get_index(self,symbol:str):
        '''
        Get the Index of the symbol from nifty 50,100,200,500
        args:
            symbol: Name /  ID od the company on NSE
        '''
        for index in self.indices.keys():
            if symbol in self.data[index]:
                return self.indices[index]
    
    
    def pick(self,budget, High = 'HIGH', Close = 'CLOSE', delta:float=0.01, nifty:str = 'nifty_500'):
        '''
        Pick Stocks based on all available and which are within your budget
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
        ''' 
        keys = set(self._eligible.keys()).intersection(set(self.data[nifty])) if nifty else list(self._eligible.keys()) 
        values = []
        for key in keys:
            df = self.open_downloaded_stock(key)
            if df.loc[0,High] + delta > budget:
                del self._eligible[key]
                
            else:
                values.append(df.iloc[0,:])
                
        columns = df.columns
        df = pd.DataFrame(values,columns=columns,index = range(len(values)))
        df = df.merge(pd.DataFrame({'SYMBOL':self._eligible.keys(), 'Diff':self._eligible.values()}),on='SYMBOL')
        
        df['Rising'] = df['SYMBOL'].apply(lambda x: self.rising[x]) # Get Rising or Falling
        df['Index'] = df['SYMBOL'].apply(lambda x: self.get_index(x)) # Get Rising or Falling
        
        self.picked = df.sort_values('Diff',ascending = True)
        return self.picked.style.apply(self.highlight_falling, column=['Rising'], axis=1) # set style
    
    
    def show_particulars(self, name, budget:float, max_loss_capacity:float, risk_to_reward_ratio:float=2, Low:str = 'LOW', High:str = 'HIGH', delta:float = 0.3, plot_candle:bool = False):
        '''
        Display the particulars of a trade before buying
        args:
            name: name of the particular stock
            loss_capacity: How much loss you can survive at the end of day PER SHARE. Total capacity will be No of shares * per share loss capacity
            risk_to_reward_ratio: How much profit you want to have. It is twice of loss_capacity per share for 44 Moving average
            budget : How much you have for investing purpose
            Low: Column name which describes LOW of the previous trade
            High =  Column name which describes High of the previous trade
            delta: A min amount above which you'll buy
            plot_candle: Plot the candlestick for the stock
        '''   
        df = self.open_downloaded_stock(name)
        if risk_to_reward_ratio > 2:
            warnings.warn(f"Don't be greedy with risk to reward ratio of {risk_to_reward_ratio}. Stick to system")
            
        if  max_loss_capacity > 0.011 * budget:
            warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total PORTFOLIO. Going Like this, you'll lose Rs {max_loss_capacity*15} in 15 Trades. Try keeping it less than 1.1% @ Rs {round(budget*0.011,2)}.")
            
        risk = max_loss_capacity # per share
        entry = df.loc[0,High] + delta # Last Day MAX + Delta
        stop_loss = min(df.loc[:2,Low].values) # Min of the last 2 
        max_loss = entry - stop_loss
        stop_loss_perc = round((max_loss / entry) *100,2)
        diff = entry - stop_loss
        quantity = min(risk // diff , budget // entry)
        profit = risk_to_reward_ratio * diff
        profit_perc = round((profit/entry)*100,2)
        target = round(entry + profit,2)
        
        
        if plot_candle:
            AnalyseStocks().plot_candlesticks(df)
    
        if budget < entry:
            warnings.warn(f"Budget for this stock should be a minimum of Rs. {entry}")
            return None
        
        if quantity < 1:
            r = round(risk + (diff - risk), 2)
            warnings.warn(f"Risk should be atleast {r} for you to afford for this stock")
            return None
            
        return {'Buying Price':round(entry,2),'Stop-Loss %': stop_loss_perc,'Target %':profit_perc,'Quantity':quantity,'Stop-Loss Price':stop_loss,'Trigger Price':target,
                'Risk Per Share':round(diff,2),'Profit Per Share':round(profit,2),'Max loss on this config':round(quantity*diff,2),
                'Max Gain on this config': round(quantity*profit,2),'Price To Profit Ratio': round(entry / (diff*2 ),2), 'Index':self.get_index(name)}
    
In = Investing()

In [21]:
In.show_particulars('FORTIS',861,40)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:86: UserWarning: You are risking 5.0% of your total PORTFOLIO. Going Like this, you'll lose Rs 600 in 15 Trades. Try keeping it less than 1.1% @ Rs 9.47.


{'Buying Price': 248.45,
 'Stop-Loss %': 4.93,
 'Target %': 9.86,
 'Quantity': 3.0,
 'Stop-Loss Price': 236.2,
 'Trigger Price': 272.95,
 'Risk Per Share': 12.25,
 'Profit Per Share': 24.5,
 'Max loss on this config': 36.75,
 'Max Gain on this config': 73.5,
 'Price To Profit Ratio': 10.14,
 'Index': 'Nifty 200'}

In [ ]:
! git add -A
! git commit -m "Ichimoku Cloud Support Added. Resonates Perfectly with the live d"

In [98]:
class Backtest():
    
    def _is_eligible():
        '''
         Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            limit: Limit difference between average and low/high threshold for selecting stocks
        ''' 
        if high < avg or close < open_: # if red candle or below Average Line, Discard
            return False
        
        diff = min(abs(low - avg), (abs(high - avg)), abs(open_ - avg), abs(close - avg))
        if diff <= limit:
            return round(diff,2)

        return False
    
    
    def _get_stats(self,df, mv:int=44, sigma:float = 0.5):
        '''
        Get Moving Average of a stock and find if that stock is in uptrend or downtrend
        args:
            df : Stocks DataFrame
            mv: Moving Average
            sigma: mv * sigma => Shows the trend. if sigma is 1, predicts trend based on 43 MV if 0.5,  predicts on past 42
        '''
        trend_limit = int(mv * sigma)
        df.sort_index(ascending=False, inplace = True)
        df[f'{str(mv)}-SMA'] = df['CLOSE'].rolling(mv, min_periods = 1).mean()
        df['Trend'] = df['44-SMA'].rolling(mv, min_periods=1).apply(lambda x: x.iloc[-trend_limit:-1].mean() < x.iloc[-1])
        df['Trend'] = df['Trend'].map({1.0:'Up', 0.0:'Down'})
        return df      

In [ ]:
def backtest(df):
    # Strating from the oldest day data + 44 -> as 44 MA is being applied
    
    # Find whether the Trend is up or down
    
    # loop from oldest + 44 and see if a stock is_eligible() and is_uptrend()
    
    # If is_eligible() and is_uptrend() -> Trigger Buy on basis of show_prticulars() -> if next day's high > buying price, Set SuccessfulBuy else save Unsuccessful

    # if in future either stoploss() or target is achieved, save that along with time, Sell Price, Loss etc
    
    

In [ ]:
def suggest(total_budget, loss_threshold, names):
    total_bearable_loss = loss_threshold * total_budget # How much % of your money, if Each trade goes wrong, you can bear
    ideal_risk = total_bearable_loss / len(names) # Ideally risk should be equally distributed in every stock
    

Total Budget = 7000
Total Risk = 700
Ideal Risk = 700 / No of available  => 700 / 13
Profit = Always twice of Risk

A: {'Buying Price': 982.25,'Risk Per Share': 12.9}
B: {'Buying Price': 883.75,'Risk Per Share': 31.05}
C: {'Buying Price': 830.25,'Risk Per Share': 26.5}
D: {'Buying Price': 678.3,'Risk Per Share': 29.0}
E: {'Buying Price': 660.8,'Risk Per Share': 21.7}
F: {'Buying Price': 552.4,'Risk Per Share': 21.4}
G: {'Buying Price': 538.3,'Risk Per Share': 13.45}
H: {'Buying Price': 432.3,'Risk Per Share': 23.8}
I: {'Buying Price': 241.1,'Risk Per Share': 9.35}
J: {'Buying Price': 222.0,'Risk Per Share': 16.9}
K: {'Buying Price': 147.65,'Risk Per Share': 5.65}
L: {'Buying Price': 143.4,'Risk Per Share': 2.85}
M: {'Buying Price': 29.75,'Risk Per Share': 0.8,}


Scenario 1:

Allocation = 7 * M
Total Risk = (800 * 29.75) => 640
Total Profit = 640 * 2 => 1280
No of Allocated = 1 {M}
Overall Risk = 1280 / 1


Scenario 2:

Allocation = (7 * B) + (1 * D) + ( 4 * M) => (7* 883.75) + (1* 678.3) + (4*29.75)
Total Risk = (7 * 31.05) + ( 1 * 29 ) + (4*0.8) => 249.549
Total Profit = 249.549 * 2 => 499.09
No of Allocated = 3 {B,D,M}
Overall risk = 499 / 3


Scenario 3:

Allocation = (7 * B) + (28*M) => (7* 883.75) + (28 * 29.75)
Total Risk = (7 * 31.05) + (28*0.8) => 239.75
Total Profit = 2 * 239.75 => 479.5
Total Allocation = 2 {B,M}
Overall risk = 479 / 2